시퀀스 데이터는 대표적으로 텍스트 데이터와 오디오 데이터가 있다.</br>
여기서는 길이가 매번 다른 시퀀스 데이터를 하나의 배치로 만드는 방법을 살펴본다.</br> 

패딩 방법은 모든 시퀀스를 가장 긴 시퀀스에 길이에 맞추고, 패딩을 채워넣는다.</br>
컴퓨터가 이해하기 쉽지만, 계산하지 않아도 될 패드부분을 계산한다는 단점이 있다.</br>
pad_sequence 함수를 사용</br>
pack_padded_sequence 함수는 padded -> packed로 변환한다.</br>

패킹 방법은 시퀀스에 길이를 저장해서 각각의 배치고 만든다. 이경우 반드시 길이 내림차순으로 정렬되어야 한다.</br>
불필요한 계산이 없고 공간을 효율적으로 쓸 수 있지만, 내림차순으로 정렬해야 하고 구현이 좀 더 복잡하다.</br> 
pack_sequence 함수를 사용</br>
pad_packed_sequence 함수는 packed -> padded로 변환한다.</br>

In [ ]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

# Random word from random word generator
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

# Make dictionary
char_set = ['<pad>'] + list(set(char for seq in data for char in seq)) # Get all characters and include pad token
char2idx = {char: idx for idx, char in enumerate(char_set)} # Constuct character to index dictionary
print('char_set:', char_set)
print('char_set length:', len(char_set))

# Convert character to index and
# Make list of tensors
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

# Check converted result
for sequence in X:
    print(sequence)

# Make length tensor (will be used later in 'pack_padded_sequence' function)
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

# Make a Tensor of shape (Batch x Maximum_Sequence_Length)
padded_sequence = pad_sequence(X, batch_first=True) # X is now padded sequence
print(padded_sequence)
print(padded_sequence.shape)

# Sort by descending lengths
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
sorted_X = [X[idx] for idx in sorted_idx]

# Check converted result
for sequence in sorted_X:
    print(sequence)

packed_sequence = pack_sequence(sorted_X)
print(packed_sequence)

# one-hot embedding using PaddedSequence
eye = torch.eye(len(char_set)) # Identity matrix of shape (len(char_set), len(char_set))
embedded_tensor = eye[padded_sequence]
print(embedded_tensor.shape) # shape: (Batch_size, max_sequence_length, number_of_input_tokens)

# one-hot embedding using PackedSequence
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

# declare RNN
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

# Try out PaddedSequence
rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape) # shape: (batch_size, max_seq_length, hidden_size)
print(hidden.shape)     # shape: (num_layers * num_directions, batch_size, hidden_size)

# Try out PackedSequence
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

# Try out pad_packed_sequence
unpacked_sequence, seq_lengths = pad_packed_sequence(embedded_packed_seq, batch_first=True)
print(unpacked_sequence.shape)
print(seq_lengths)

# Construct embedded_padded_sequence
embedded_padded_sequence = eye[pad_sequence(sorted_X, batch_first=True)]
print(embedded_padded_sequence.shape)

# Try out pack_padded_sequence
sorted_lengths = sorted(lengths, reverse=True)
new_packed_sequence = pack_padded_sequence(embedded_padded_sequence, sorted_lengths, batch_first=True)
print(new_packed_sequence.data.shape)
print(new_packed_sequence.batch_sizes)
